In [ ]:
from pathlib import Path
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from segmentflow import segment, view
from skimage import morphology, util
%load_ext autoreload
%autoreload 2

In [ ]:
img_dir = Path(r'..\data\IP_01-he6_raw')
# img_dir = Path(r'..\data\IDOX_Estane test 1 TIFF\Load 0')
file_suffix = '.tiff'
print(len([p for p in img_dir.glob(f'*{file_suffix}')]))
imgs = segment.load_images(
    img_dir,
    slice_crop=[0, 500],
    row_crop=[200, 850],
    col_crop=[200, 850],
    file_suffix=file_suffix
)
fig, axes = view.vol_slices(imgs, nslices=5)
plt.show()

In [ ]:
# Plot intensity rescale histogram
imgs = segment.preprocess(
    imgs, median_filter=True,
    rescale_intensity_range=[0.01, 99.99])
imgs = util.img_as_uint(imgs)
fig, axes = view.vol_slices(imgs, nslices=5)
plt.show()

In [ ]:
# Calc semantic seg threshold values and generate histogram
thresholds, fig, ax = segment.threshold_multi_otsu(
    imgs, nclasses=2, nbins=256, return_fig_ax=True
)
plt.show()

In [ ]:
imgs = segment.isolate_classes(imgs, thresholds)
fig, axes = view.vol_slices(imgs, nslices=5)
plt.show()

In [ ]:
fig, axes = view.vol_slices(imgs.astype(bool), nslices=5)
plt.show()

In [ ]:
imgs_shell = imgs - morphology.binary_erosion(imgs)
fig, axes = view.vol_slices(imgs_shell, nslices=5)
plt.show()

In [ ]:
fig, axes = view.vol_slices(imgs_shell[:, 300:400, 300:400], nslices=5)
plt.show()

In [ ]:
shell_verts = np.argwhere(imgs_shell == 1)
shell_verts

In [ ]:
shell_df = pd.DataFrame(shell_verts, columns=['slice', 'row', 'column'])
shell_df.to_csv(f'../results/{img_dir.name}_shel.csv')
shell_df.head()

In [ ]:
print('Total n voxels:', math.prod(i for i in imgs.shape))
print('Shell n vertices:', shell_df.index.shape[0])